In [0]:
df=spark.read.format("delta").load("s3://silver-layerbucket-sd/cleaned_data/")

In [0]:
display(df)

In [0]:
# To count number of rows in the dataframe
row_count = df.count()
display(row_count)

In [0]:
# To count number of columns in the dataframe
column_count = len(df.columns)
display(column_count)


In [0]:
from pyspark.sql.functions import *
daily_sales = df.groupby("Date").agg(count('*').alias('Total_Orders'),
                                           sum("Qty").alias("Total_Quantity"),
                                           sum("Amount").alias("Total_Revenue"))

In [0]:
display(daily_sales)

In [0]:
daily_sales.write.mode("append").format("delta").save("s3://gold-layer-sd/goldlayer_daily_sales_performance/")

In [0]:
# Statewise revenue
from pyspark.sql.functions import sum

state_wise_revenue = (
    df.groupBy("ship_state")
      .agg(sum("Amount").alias("total_revenue"))
)

state_wise_revenue.show()


In [0]:
state_wise_revenue.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("s3://gold-layer-sd/gold_layer_statewise_revenue/")

In [0]:
#citywise revenue
from pyspark.sql.functions import sum

city_wise_revenue = (
    df.groupBy("ship_city")
      .agg(sum("Amount").alias("total_revenue"))
)

display(city_wise_revenue)


In [0]:
city_wise_revenue.write.mode("append").format("csv").save("s3://gold-layer-sd/gold_layer_citywise_revenue/")

In [0]:
# status_wise_revenue
from pyspark.sql.functions import sum

status_wise_revenue = (
    df.groupBy("Status")
      .agg(sum("Amount").alias("total_revenue"))
)

display(status_wise_revenue)


In [0]:
status_wise_revenue.write.mode("append").format("delta").save("s3://gold-layer-sd/gold_layer_statuswise_revenue/")

In [0]:
# Category wise total quantity
from pyspark.sql.functions import sum

category_wise_qty = (
    df.groupBy("Category")
      .agg(sum("Qty").alias("total_quantity"))
)

category_wise_qty.show()


In [0]:
category_wise_qty.write.mode("append").format("delta").save("s3://gold-layer-sd/goldlayer_Category_wise_quantity/")

In [0]:
df.write.mode("append").format("delta").save("s3://gold-layer-sd/Gold_layer_all_data/")